### 操作方式跟dataclass一樣

In [6]:
from pydantic import BaseModel

class Person(BaseModel):
    # field一定要有type hint
    first_name: str
    last_name: str
    age: int

p = Person(first_name='Alan', last_name='Chester', age=24)
print(p.first_name)
print(p.last_name)
print(p.age)
print(p)

Alan
Chester
24
first_name='Alan' last_name='Chester' age=24


### dataclass做不到的
- 驗證資料

In [8]:
p1 = Person(first_name='Alan', last_name='Chester', age='24') # 自動轉換型別
print(p1)

first_name='Alan' last_name='Chester' age=24


In [10]:
try:
    p2 = Person(first_name='Alan', last_name='Chester', age='abc') # 自動轉換型別
except Exception as error:
    print(error)

1 validation error for Person
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='abc', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/int_parsing


### 可以將純文字轉成pydantic實體
- model_validate_json()

In [12]:
data_json = '''
{
    "first_name": "Alan",
    "last_name": "Chester",
    "age": 24
}
'''
p3 = Person.model_validate_json(data_json)
print(p3)

first_name='Alan' last_name='Chester' age=24


### 可以選擇需要的欄位

In [14]:
data_json = '''
{
    "first_name": "Alan",
    "last_name": "Chester",
    "age": 24,
    "height": 170
}
'''
p4 = Person.model_validate_json(data_json)
print(p4)

first_name='Alan' last_name='Chester' age=24


### 可以設定取得的資料，如果沒有這個欄位，設定欄位預設值

In [15]:
class Person(BaseModel):
    # field一定要有type hint
    first_name: str # 一定要有資料
    last_name: str # 一定要有資料
    age: int = 0 # 有預設的值

data_json = '''
{
    "first_name": "Alan",
    "last_name": "Chester"
}
'''
p4 = Person.model_validate_json(data_json)
print(p4)

first_name='Alan' last_name='Chester' age=0


In [16]:
class Person(BaseModel):
    # field一定要有type hint
    first_name: str # 一定要有資料
    last_name: str # 一定要有資料
    age: int = 0 # 有預設的值
    lucky_numbers: list[int] = []

data_json = '''
{
    "first_name": "Alan",
    "last_name": "Chester"
}
'''
p4 = Person.model_validate_json(data_json)
print(p4)

first_name='Alan' last_name='Chester' age=0 lucky_numbers=[]


In [17]:
class Person(BaseModel):
    # field一定要有type hint
    first_name: str # 一定要有資料
    last_name: str # 一定要有資料
    age: int = 0 # 有預設的值
    lucky_numbers: list[int] = []

data_json = '''
{
    "first_name": "Alan",
    "last_name": "Chester",
    "lucky_numbers": [1, 2, 3, 4, 5]
}
'''
p4 = Person.model_validate_json(data_json)
print(p4)

first_name='Alan' last_name='Chester' age=0 lucky_numbers=[1, 2, 3, 4, 5]


In [19]:
data_json = '''
{
    "id": 100,
    "First Name": "Alan",
    "LASTNAME": "Chester",
    "age in years": 24
}
'''

from pydantic import BaseModel, Field

class Person(BaseModel):
    id_: int = Field(alias='id')
    first_name: str = Field(alias='First Name')
    last_name: str = Field(alias='LASTNAME')
    age: int = Field(alias='age in years', default=0)

p1 = Person.model_validate_json(data_json)
print(p1)

id_=100 first_name='Alan' last_name='Chester' age=24


### 巢狀的Model

In [23]:
data: str = '''
{
    "firstName": "Arthur",
    "lastName": "Clarke",
    "born": {
        "place": {
            "country": "Lunar Colony",
            "city": "Central City"            
        },
        "date": "2001-01-01"
    }
}
'''

from pydantic import BaseModel, Field

class Place(BaseModel):
    country: str
    city: str

class Born(BaseModel):
    place: Place
    date: str

class Person(BaseModel):
    first_name: str = Field(alias='firstName')
    last_name: str = Field(alias='lastName')
    born: Born

p1 = Person.model_validate_json(data)
print(p1.born.place.country)
print(p1.born.place.city)

Lunar Colony
Central City
